In [0]:
sc

<SparkContext master=spark://10.139.64.4:7077 appName=Databricks Shell>

In [0]:
spark

In [0]:
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load("dbfs:/Volumes/azuredatabricks/default/sales/zomato_orders_large.csv"
)

In [0]:
df.show()

+-------+----------+--------------+----------------+--------+-----+----------+
|OrderID|CustomerID|RestaurantName|        DishName|Quantity|Price| OrderDate|
+-------+----------+--------------+----------------+--------+-----+----------+
|      1|       183|     Spice Hub|    Paneer Tikka|       3|  322|2024-06-04|
|      2|       186|   Sushi World|    Cheeseburger|       5|  466|2024-06-23|
|      3|       127|   Pasta Place|    Cheeseburger|       5|  260|2024-06-08|
|      4|       145|  Pizza Palace|  Mango Smoothie|       2|  382|2024-06-26|
|      5|       144|  Pizza Palace| Pepperoni Pizza|       1|  154|2024-06-26|
|      6|       110|  Pizza Palace|    Paneer Tikka|       1|  411|2024-06-18|
|      7|       156|   Pasta Place|    Cheeseburger|       1|  110|2024-06-05|
|      8|       126|   Sushi World|  Veggie Delight|       5|  494|2024-06-05|
|      9|       108|  Burger Haven| Pepperoni Pizza|       3|  474|2024-06-02|
|     10|       119|   Pasta Place| Pepperoni Pizza|

In [0]:
df.cache()

DataFrame[OrderID: int, CustomerID: int, RestaurantName: string, DishName: string, Quantity: int, Price: int, OrderDate: date]

In [0]:
df.printSchema()

root
 |-- OrderID: integer (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- RestaurantName: string (nullable = true)
 |-- DishName: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- OrderDate: date (nullable = true)



In [0]:
from pyspark.sql.functions import *
df.select([count(when(isnull(col), col)).alias(col) for col in df.columns]).show()

+-------+----------+--------------+--------+--------+-----+---------+
|OrderID|CustomerID|RestaurantName|DishName|Quantity|Price|OrderDate|
+-------+----------+--------------+--------+--------+-----+---------+
|      0|         0|             0|       0|       0|    0|        0|
+-------+----------+--------------+--------+--------+-----+---------+



In [0]:
df = df.na.drop()

In [0]:
from pyspark.sql.functions import *
df.select([count(when(isnull(col), col)).alias(col) for col in df.columns]).show()

+-------+----------+--------------+--------+--------+-----+---------+
|OrderID|CustomerID|RestaurantName|DishName|Quantity|Price|OrderDate|
+-------+----------+--------------+--------+--------+-----+---------+
|      0|         0|             0|       0|       0|    0|        0|
+-------+----------+--------------+--------+--------+-----+---------+



In [0]:
df = df.dropDuplicates()

In [0]:
df.show()

+-------+----------+--------------+-------------------+--------+-----+----------+
|OrderID|CustomerID|RestaurantName|           DishName|Quantity|Price| OrderDate|
+-------+----------+--------------+-------------------+--------+-----+----------+
|     84|       186|     Spice Hub|      Chicken Curry|       2|  305|2024-06-24|
|    639|       173|     Spice Hub|    Pepperoni Pizza|       2|  297|2024-06-10|
|    782|       140|     Spice Hub|   Margherita Pizza|       4|  149|2024-06-06|
|    797|       150|  Burger Haven|       Paneer Tikka|       2|  159|2024-06-19|
|     14|       113|  Pizza Palace|      Chicken Curry|       3|  380|2024-06-07|
|    316|       178|   Sushi World|       Paneer Tikka|       3|  221|2024-06-08|
|    318|       115|     Spice Hub|   Margherita Pizza|       1|  457|2024-06-15|
|    543|       181|  Pizza Palace|    California Roll|       1|  430|2024-06-18|
|    612|       129|  Burger Haven|    Pepperoni Pizza|       3|  143|2024-06-15|
|    743|       

In [0]:
df = df.withColumn("month", month("OrderDate"))
df.show()

+-------+----------+--------------+-------------------+--------+-----+----------+-----+
|OrderID|CustomerID|RestaurantName|           DishName|Quantity|Price| OrderDate|month|
+-------+----------+--------------+-------------------+--------+-----+----------+-----+
|     84|       186|     Spice Hub|      Chicken Curry|       2|  305|2024-06-24|    6|
|    639|       173|     Spice Hub|    Pepperoni Pizza|       2|  297|2024-06-10|    6|
|    782|       140|     Spice Hub|   Margherita Pizza|       4|  149|2024-06-06|    6|
|    797|       150|  Burger Haven|       Paneer Tikka|       2|  159|2024-06-19|    6|
|     14|       113|  Pizza Palace|      Chicken Curry|       3|  380|2024-06-07|    6|
|    316|       178|   Sushi World|       Paneer Tikka|       3|  221|2024-06-08|    6|
|    318|       115|     Spice Hub|   Margherita Pizza|       1|  457|2024-06-15|    6|
|    543|       181|  Pizza Palace|    California Roll|       1|  430|2024-06-18|    6|
|    612|       129|  Burger Hav

In [0]:
# from pyspark.ml.feature import StringIndexer
# indexer = StringIndexer(inputCol='Month',outputCol='Month_index')
# # fit string indexer model on the dataframe
# df1 = indexer.fit(df).transform(df)
# df1.show()

In [0]:
# df2 = df1.drop('Month')

In [0]:
df.createOrReplaceTempView("zomato")

 Total Revenue per Restaurant

In [0]:
%sql
SELECT RestaurantName, SUM(Quantity * Price) AS TotalRevenue
FROM zomato
GROUP BY RestaurantName
ORDER BY TotalRevenue DESC;

RestaurantName,TotalRevenue
Sushi World,197839
Burger Haven,193339
Spice Hub,171724
Pasta Place,171008
Pizza Palace,166896


Most Popular Dish by Restaurant

In [0]:
%sql
SELECT RestaurantName, DishName, SUM(Quantity) AS TotalOrdered
FROM zomato
GROUP BY RestaurantName, DishName
ORDER BY RestaurantName, TotalOrdered DESC;

RestaurantName,DishName,TotalOrdered
Burger Haven,Cheeseburger,90
Burger Haven,Veggie Delight,75
Burger Haven,Chicken Curry,75
Burger Haven,Spaghetti Carbonara,74
Burger Haven,Margherita Pizza,61
Burger Haven,Mango Smoothie,58
Burger Haven,California Roll,57
Burger Haven,Paneer Tikka,54
Burger Haven,Chicken Biryani,54
Burger Haven,Pepperoni Pizza,47


Top 5 Customers by Order Count

In [0]:
%sql
SELECT CustomerID, COUNT(OrderID) AS OrderCount
FROM zomato
GROUP BY CustomerID
ORDER BY OrderCount DESC
LIMIT 5;

CustomerID,OrderCount
130,17
111,17
117,16
177,16
123,16


Total Orders and Revenue by Month



In [0]:
%sql
SELECT month, COUNT(OrderID) AS TotalOrders, SUM(Quantity * Price) AS TotalRevenue
FROM zomato
GROUP BY month
ORDER BY month;

month,TotalOrders,TotalRevenue
6,1000,900806


 Average Order Value by Restaurant

In [0]:
%sql
SELECT RestaurantName, AVG(Quantity * Price) AS AvgOrderValue
FROM zomato
GROUP BY RestaurantName
ORDER BY AvgOrderValue DESC;



RestaurantName,AvgOrderValue
Burger Haven,952.4088669950739
Sushi World,915.9212962962963
Spice Hub,894.3958333333334
Pizza Palace,892.4919786096257
Pasta Place,846.5742574257425


Find restaurants whose average revenue per order is above the overall average.

In [0]:
%sql
SELECT RestaurantName, AVG(Quantity * Price) AS AvgRevenuePerOrder
FROM zomato
GROUP BY RestaurantName
HAVING AVG(Quantity * Price) > (
    SELECT AVG(OrderRevenue) 
    FROM (
        SELECT (Quantity * Price) AS OrderRevenue
        FROM zomato
    ) AS OrderRevenues
)
ORDER BY AvgRevenuePerOrder DESC;


RestaurantName,AvgRevenuePerOrder
Burger Haven,952.4088669950739
Sushi World,915.9212962962963


Finding the Most Recent Order for Each Customer

In [0]:
%sql
WITH RecentOrders AS (
    SELECT CustomerID, MAX(OrderDate) AS MostRecentOrderDate
    FROM zomato
    GROUP BY CustomerID
)
SELECT o.CustomerID, o.OrderID, o.RestaurantName, o.DishName, o.Quantity, o.Price, o.OrderDate
FROM zomato o
JOIN RecentOrders r ON o.CustomerID = r.CustomerID AND o.OrderDate = r.MostRecentOrderDate;


CustomerID,OrderID,RestaurantName,DishName,Quantity,Price,OrderDate
191,565,Pasta Place,Cheeseburger,3,110,2024-06-27
183,336,Sushi World,Chicken Biryani,5,362,2024-06-21
120,42,Burger Haven,Paneer Tikka,5,491,2024-06-28
135,916,Pasta Place,California Roll,2,451,2024-06-20
168,862,Sushi World,Cheeseburger,1,190,2024-06-28
188,300,Spice Hub,Spaghetti Carbonara,3,204,2024-06-26
196,479,Spice Hub,Cheeseburger,1,125,2024-06-28
176,243,Spice Hub,Paneer Tikka,1,165,2024-06-27
127,287,Spice Hub,Chicken Biryani,4,171,2024-06-27
114,633,Burger Haven,Chicken Curry,1,363,2024-06-28
